In [62]:
import numpy as np
import pandas as pd

from linearmodels import PanelOLS, RandomEffects
import datetime
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [63]:
admin1_O = pd.read_csv('model_dataset_start.csv', index_col=0)

admin1_O['date_1'] = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in admin1_O['date']]

admin1_O.columns

Index(['date', 'id_num', 'trip_num', 'admin1Name_1', 'rwi_pctl',
       'rwi_ctwt_pctl', 'trips/person', 'trip_miles/person', 'miles/person',
       '0_14_pct', '15_64_pct', '0_19_pct', '20_39_pct', '40_64_pct',
       '65p_pct', 'M_pct', 'Population', 'health_fclt_count',
       'admin1_infected', 'first_case', 'ban_travel', 'fed_lockdown_p1',
       'fed_lockdown_p2', 'inter_state_travel_ban', 'date_1'],
      dtype='object')

### adjust the scale of each variable


In [64]:
# adjust the scale of each variable
admin1_O['health_fclt_count_1K'] = admin1_O['health_fclt_count'] / 1000
admin1_O['0_14_pct'] = admin1_O['0_14_pct'] * 100
admin1_O['15_64_pct'] = admin1_O['15_64_pct'] * 100
admin1_O['65p_pct'] = admin1_O['65p_pct'] * 100
admin1_O['M_pct'] = admin1_O['M_pct'] * 100

admin1_O['trip_kms/person'] = admin1_O['trip_miles/person'] * 1.609344

### shift for lag 1


In [65]:
temp_lst = []
for st in admin1_O['admin1Name_1'].drop_duplicates():
    temp = admin1_O.loc[admin1_O['admin1Name_1'] == st]
    temp['l1_trips/person'] = temp['trips/person'].shift(1)
    temp['l1_trip_miles/person'] = temp['trip_miles/person'].shift(1)
    temp['l1_trip_kms/person'] = temp['trip_kms/person'].shift(1)
    temp = temp.dropna()
    temp_lst.append(temp)
    # break
admin1_O = pd.concat(temp_lst, axis=0)
admin1_O

c:\Users\k5coo\anaconda3\envs\trsp\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\k5coo\anaconda3\envs\trsp\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\k5coo\anaconda3\envs\trsp\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,date,id_num,trip_num,admin1Name_1,rwi_pctl,rwi_ctwt_pctl,trips/person,trip_miles/person,miles/person,0_14_pct,...,ban_travel,fed_lockdown_p1,fed_lockdown_p2,inter_state_travel_ban,date_1,health_fclt_count_1K,trip_kms/person,l1_trips/person,l1_trip_miles/person,l1_trip_kms/person
37,2020-01-02,45,89,Bauchi,0.135135,0.081081,1.977778,6.206332,50.092951,48.096824,...,0,0,0,0,2020-01-02,1.312,9.988123,2.351351,6.258242,10.071664
74,2020-01-03,26,64,Bauchi,0.135135,0.081081,2.461538,6.435228,64.674084,48.096824,...,0,0,0,0,2020-01-03,1.312,10.356495,1.977778,6.206332,9.988123
111,2020-01-04,38,64,Bauchi,0.135135,0.081081,1.684211,18.228955,52.410111,48.096824,...,0,0,0,0,2020-01-04,1.312,29.336660,2.461538,6.435228,10.356495
148,2020-01-05,50,97,Bauchi,0.135135,0.081081,1.940000,6.821064,52.142714,48.096824,...,0,0,0,0,2020-01-05,1.312,10.977439,1.684211,18.228955,29.336660
185,2020-01-06,50,104,Bauchi,0.135135,0.081081,2.080000,4.464782,14.144533,48.096824,...,0,0,0,0,2020-01-06,1.312,7.185370,1.940000,6.821064,10.977439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,2020-04-21,91,186,Zamfara,0.081081,0.108108,2.043956,7.206081,17.225763,48.229513,...,1,1,1,0,2020-04-21,0.908,11.597063,2.128713,8.173227,13.153533
4180,2020-04-22,85,155,Zamfara,0.081081,0.108108,1.823529,7.828871,24.630431,48.229513,...,1,1,1,0,2020-04-22,0.908,12.599347,2.043956,7.206081,11.597063
4217,2020-04-23,80,158,Zamfara,0.081081,0.108108,1.975000,8.173407,22.628134,48.229513,...,1,1,1,1,2020-04-23,0.908,13.153824,1.823529,7.828871,12.599347
4254,2020-04-24,39,66,Zamfara,0.081081,0.108108,1.692308,3.350973,8.065026,48.229513,...,1,1,1,1,2020-04-24,0.908,5.392868,1.975000,8.173407,13.153824


In [66]:
admin1_O.describe()

,id_num,trip_num,rwi_pctl,rwi_ctwt_pctl,trips/person,trip_miles/person,miles/person,0_14_pct,15_64_pct,0_19_pct,...,first_case,ban_travel,fed_lockdown_p1,fed_lockdown_p2,inter_state_travel_ban,health_fclt_count_1K,trip_kms/person,l1_trips/person,l1_trip_miles/person,l1_trip_kms/person
count,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,...,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000
mean,1128.853819,2275.003995,0.513514,0.513514,2.047208,7.708610,34.993279,41.879285,54.879623,0.529490,...,0.513043,0.330435,0.234783,0.113043,0.026087,1.246243,12.405805,2.047335,7.735721,12.449437
std,3456.184537,6648.133777,0.288604,0.288604,0.404733,4.887938,21.983193,5.113548,4.658226,0.046654,...,0.499889,0.470425,0.423913,0.316683,0.159413,0.483943,7.866374,0.404326,4.865450,7.830183
min,4.000000,8.000000,0.027027,0.027027,1.235294,1.440347,3.320644,33.020089,48.801456,0.428685,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.385000,2.318013,1.235294,1.440347,2.318013
25%,108.000000,220.000000,0.270270,0.270270,1.770795,5.381419,22.535130,37.144735,50.489950,0.486016,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.893000,8.660555,1.771170,5.410607,8.707527
50%,233.000000,481.000000,0.513514,0.513514,1.989899,6.806025,30.648538,43.856274,53.000453,0.544705,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.149000,10.953235,1.987952,6.823951,10.982084
75%,693.500000,1318.000000,0.756757,0.756757,2.243926,8.544290,40.694348,46.913882,59.331451,0.574741,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.562000,13.750702,2.243803,8.564137,13.782643
max,30469.000000,57011.000000,1.000000,1.000000,4.255319,103.488975,268.530050,48.655804,64.471120,0.588718,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.331000,166.549362,4.255319,103.488975,166.549362


## model for trip per person, for all states

In [67]:
model_cols = [
    'date_1', 
    'admin1Name_1', 
    'rwi_pctl', 
    # 'rwi_ctwt_pctl', 
    'trips/person', 
    'l1_trips/person', 
    # 'trip_miles/person', 
    # 'l1_trip_miles/person', 
    'trip_kms/person', 
    'l1_trip_kms/person', 
    '0_14_pct', 
    # '15_64_pct', 
    '65p_pct', 
    'M_pct', 
    'health_fclt_count_1K', 
    'admin1_infected', 
    'first_case', 
    'ban_travel', 
    'fed_lockdown_p1', 
    'fed_lockdown_p2', 
]
admin1_O[model_cols].describe()

,rwi_pctl,trips/person,l1_trips/person,trip_kms/person,l1_trip_kms/person,0_14_pct,65p_pct,M_pct,health_fclt_count_1K,admin1_infected,first_case,ban_travel,fed_lockdown_p1,fed_lockdown_p2
count,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000
mean,0.513514,2.047208,2.047335,12.405805,12.449437,41.879285,3.241092,50.814258,1.246243,0.277321,0.513043,0.330435,0.234783,0.113043
std,0.288604,0.404733,0.404326,7.866374,7.830183,5.113548,0.734903,0.945853,0.483943,2.881729,0.499889,0.470425,0.423913,0.316683
min,0.027027,1.235294,1.235294,2.318013,2.318013,33.020089,1.481091,48.606968,0.385000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.270270,1.770795,1.771170,8.660555,8.707527,37.144735,2.639501,50.329965,0.893000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.513514,1.989899,1.987952,10.953235,10.982084,43.856274,3.143273,50.681143,1.149000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.756757,2.243926,2.243803,13.750702,13.782643,46.913882,3.784668,51.084448,1.562000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,4.255319,4.255319,166.549362,166.549362,48.655804,4.720355,53.202701,2.331000,80.000000,1.000000,1.000000,1.000000,1.000000


In [68]:
mdata = admin1_O[model_cols].set_index(['admin1Name_1', 'date_1'])

In [69]:
mdata.describe()

,rwi_pctl,trips/person,l1_trips/person,trip_kms/person,l1_trip_kms/person,0_14_pct,65p_pct,M_pct,health_fclt_count_1K,admin1_infected,first_case,ban_travel,fed_lockdown_p1,fed_lockdown_p2
count,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000,4255.000000
mean,0.513514,2.047208,2.047335,12.405805,12.449437,41.879285,3.241092,50.814258,1.246243,0.277321,0.513043,0.330435,0.234783,0.113043
std,0.288604,0.404733,0.404326,7.866374,7.830183,5.113548,0.734903,0.945853,0.483943,2.881729,0.499889,0.470425,0.423913,0.316683
min,0.027027,1.235294,1.235294,2.318013,2.318013,33.020089,1.481091,48.606968,0.385000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.270270,1.770795,1.771170,8.660555,8.707527,37.144735,2.639501,50.329965,0.893000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.513514,1.989899,1.987952,10.953235,10.982084,43.856274,3.143273,50.681143,1.149000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.756757,2.243926,2.243803,13.750702,13.782643,46.913882,3.784668,51.084448,1.562000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,4.255319,4.255319,166.549362,166.549362,48.655804,4.720355,53.202701,2.331000,80.000000,1.000000,1.000000,1.000000,1.000000


### fit trips model

In [70]:
y_cols = ['trips/person']

# exclude some columns
ex_cols = [
 'trip_kms/person',
  'M_pct',
           ]
x_cols = [x for x in list(mdata.columns) if x not in y_cols + ex_cols]

x_cols

['rwi_pctl',
 'l1_trips/person',
 'l1_trip_kms/person',
 '0_14_pct',
 '65p_pct',
 'health_fclt_count_1K',
 'admin1_infected',
 'first_case',
 'ban_travel',
 'fed_lockdown_p1',
 'fed_lockdown_p2']

#### random effects

In [71]:
exog = sm.add_constant(mdata[x_cols])
mod = RandomEffects(mdata[y_cols[0]], exog)
# res = mod.fit(cov_type='clustered', cluster_entity=True)
rd_res = mod.fit()
print(rd_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:           trips/person   R-squared:                        0.6622
Estimator:              RandomEffects   R-squared (Between):              0.9649
No. Observations:                4255   R-squared (Within):               0.2951
Date:                Thu, Apr 06 2023   R-squared (Overall):              0.6622
Time:                        00:30:21   Log-likelihood                    120.54
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      756.09
Entities:                          37   P-value                           0.0000
Avg Obs:                       115.00   Distribution:                 F(11,4243)
Min Obs:                       115.00                                           
Max Obs:                       115.00   F-statistic (robust):             756.09
                            

### fit distance model

In [72]:
y_cols = ['trip_kms/person']
# y_cols = ['trip_miles/person']

# exclude some columns
ex_cols = [
'trips/person', 
 'M_pct',
           ]
x_cols = [x for x in list(mdata.columns) if x not in y_cols + ex_cols]

x_cols

['rwi_pctl',
 'l1_trips/person',
 'l1_trip_kms/person',
 '0_14_pct',
 '65p_pct',
 'health_fclt_count_1K',
 'admin1_infected',
 'first_case',
 'ban_travel',
 'fed_lockdown_p1',
 'fed_lockdown_p2']

#### random effects

In [73]:
exog = sm.add_constant(mdata[x_cols])
mod = RandomEffects(mdata[y_cols[0]], exog)
# res = mod.fit(cov_type='clustered', cluster_entity=True)
rd_res = mod.fit()
print(rd_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:        trip_kms/person   R-squared:                        0.2142
Estimator:              RandomEffects   R-squared (Between):              0.7298
No. Observations:                4255   R-squared (Within):               0.0413
Date:                Thu, Apr 06 2023   R-squared (Overall):              0.2142
Time:                        00:30:21   Log-likelihood                 -1.43e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      105.17
Entities:                          37   P-value                           0.0000
Avg Obs:                       115.00   Distribution:                 F(11,4243)
Min Obs:                       115.00                                           
Max Obs:                       115.00   F-statistic (robust):             105.17
                            